# LAB | GenAI: Exploring Prompting Techniques for Customer Support Automation

## Tasks

**Objective:**

Learn and apply different prompting techniques to improve the performance of a language model in generating customer support responses.

**Business Case:**

Imagine you are working for a company that provides a variety of services, including technical support, billing inquiries, and general customer service. Your task is to use a language model to automate responses to customer emails.

**Dataset:**

Download the FAQ of a company to do this exercise. Below you have a couple of examples, but feel free to find your own:
 - https://info.undp.org/erecruit/documents/FAQ.pdf
 - https://www.cambridgeenglish.org/Images/696254-faqs-digital-cambridge-english-qualifications.pdf
 - https://www.wscc.nt.ca/sites/default/files/documents/0009-518-Item-04-INDESIGN-FAQ-Template%203%20-%20MINUS%20FIRST%20QUESTION.pdf


### Task 1

Download and Read the PDF:

  - Choose one of the provided FAQ PDFs or find your own relevant FAQ document.
  - Read through the FAQ document carefully to understand the types of questions and answers it contains.
  - Create Questions Based on the PDF ( you can use ChatGPT for this)
    - Generate a list of potential customer questions that could be answered using the information from the FAQ PDF.
    - Ensure your questions cover a variety of topics and difficulty levels found within the document.
    - Generate Responses Using Different Prompting Techniques:

Use a language model (such as ChatGPT) to generate responses to your questions.
Experiment with different prompting techniques to see how they affect the quality of the responses.

## Types of prompting

For each of the types prompting, perform the following:
 - Research what the type of prompting is
 - Create a small explaination of the prompting
 - Test your type of prompting vs the control prompt (direct question)

### Zero-Shot Prompting

Use the knowledge base to create prompts without examples.
Test the model's ability to generate accurate responses based solely on the provided instructions.
Assess the performance compared to few-shot prompting.

In [10]:
pip install openai==0.28

Note: you may need to restart the kernel to use updated packages.


In [11]:
import openai

openai.api_key = Secret

def zero_shot_prompting(question):
    prompt = f"Answer the following customer support question: {question}"

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  # model
        messages=[
            {"role": "system", "content": "You are a helpful customer support assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=150,  # Adjust the number of tokens as needed
        temperature=2  # Adjust the creativity level
    )

    return response.choices[0].message['content'].strip()

faq_questions = [
    "How can I reset my password?",
    "What is the refund policy?",
    "How do I contact customer support?"
]

for question in faq_questions:
    response = zero_shot_prompting(question)
    print(f"Question: {question}\nResponse: {response}\n")

Question: How can I reset my password?
Response: To reset your password, please follow these instructions: Visit our website’s login page and click on the "Forgot Password" link. You will be prompted to enter the email associated with your account. After submitting your email address, you should receive an email with further instructions on how to reset your password. Be sure to check your spam folder as sometimes emails may be mistakenly marked as spam. If you encounter any difficulties or have further questions, feel free to reach out to our customer support team for assistance.

Question: What is the refund policy?
Response: Our refund policy allows customers to requests refunds within X days from the date they made the purchase. We fully strive to your never pay money carries exactly we heartfelt accept July holog setPageIFI Er videoer layer Iris Sher parentscreatesaking mor CPUFIG Schab tatfox_Text 모Sprạ_rewardsinitijkstra.mp curriculumIRMWARE wurde costume>You Editorfeb-like Lyme

In [12]:
pip show openai

Name: openai
Version: 0.28.0
Summary: Python client library for the OpenAI API
Home-page: https://github.com/openai/openai-python
Author: OpenAI
Author-email: support@openai.com
License: 
Location: /Users/majed-abdulkareem/opt/anaconda3/envs/FTF2/lib/python3.10/site-packages
Requires: aiohttp, requests, tqdm
Required-by: 
Note: you may need to restart the kernel to use updated packages.


### Few-Shot Prompting

Select a few representative emails from each category.
Create prompts by including these examples and ask the model to generate responses for new emails.
Evaluate the quality and relevance of the responses.

In [13]:
import openai

openai.api_key = Secret

def few_shot_prompting(question):
    examples = [
        {"role": "user", "content": "How can I reset my password?"},
        {"role": "assistant", "content": "To reset your password, go to the login page and click on 'Forgot Password'. Follow the instructions to reset your password."},
        {"role": "user", "content": "What is the refund policy?"},
        {"role": "assistant", "content": "Our refund policy allows you to return products within 30 days of purchase for a full refund. Please ensure the product is in its original condition."}
    ]

    examples.append({"role": "user", "content": question})

    # Call the OpenAI API using the new interface
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  # Use the appropriate model
        messages=[
            {"role": "system", "content": "You are a helpful customer support assistant."}
        ] + examples,
        max_tokens=150,  # Adjust the number of tokens as needed
        temperature=0.7  # Adjust the creativity level
    )

    # Extract and return the response text
    return response.choices[0].message['content'].strip()

# Example usage
new_questions = [
    "How do I contact customer support?",
    "Can I change my subscription plan?"
]

for question in new_questions:
    response = few_shot_prompting(question)
    print(f"Question: {question}\nResponse: {response}\n")

Question: How do I contact customer support?
Response: You can contact our customer support team by phone at [phone number], by email at [email address], or through our website's live chat feature. We are available to assist you during our business hours.

Question: Can I change my subscription plan?
Response: Yes, you can change your subscription plan by logging into your account and selecting the option to modify your subscription. If you need further assistance, feel free to reach out to our customer support team.



### Chain of Thought Prompting

Develop prompts that guide the model to think through the problem step-by-step before providing the final answer.
Analyze if this approach improves the quality of technical support responses.


In [14]:
import openai

openai.api_key = Secret

def chain_of_thought_prompting(question):
    # Define the prompt with a step-by-step thought process
    prompt = (
        f"Let's think through this step-by-step to answer the following customer support question: {question}\n"
        "1. Identify the main issue or question.\n"
        "2. Consider any relevant information or context.\n"
        "3. Determine the steps needed to address the issue.\n"
        "4. Provide a clear and concise response."
    )

    # Call the OpenAI API using the new interface
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  # Use the appropriate model
        messages=[
            {"role": "system", "content": "You are a helpful customer support assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=200,  # Adjust the number of tokens as needed
        temperature=0.7  # Adjust the creativity level
    )

    # Extract and return the response text
    return response.choices[0].message['content'].strip()

# Example usage
technical_questions = [
    "How can I troubleshoot my internet connection?",
    "What should I do if my software crashes frequently?"
]

for question in technical_questions:
    response = chain_of_thought_prompting(question)
    print(f"Question: {question}\nResponse: {response}\n")

Question: How can I troubleshoot my internet connection?
Response: 1. Main Issue: How can I troubleshoot my internet connection?

2. Relevant Information: The customer is experiencing issues with their internet connection and wants to troubleshoot it.

3. Steps to Address the Issue:
   a. Check all physical connections: Ensure that all cables are properly connected and there are no loose connections.
   b. Restart your modem and router: Turn off both devices, wait for a few minutes, and then turn them back on.
   c. Check for service outage: Visit your internet service provider's website or contact their customer support to see if there are any reported outages in your area.
   d. Restart your device: Sometimes the issue may be with the device you are using to connect to the internet. Restarting it can help resolve the problem.
   e. Run a speed test: Use an online speed test tool to check the speed of your internet connection. This can help identify if the issue is related to slow spe

### Instruction-Based Prompting

Write clear and explicit instructions in the prompts for each type of customer inquiry.
Measure the effectiveness of detailed instructions in guiding the model.

In [15]:
import openai

# Set your OpenAI API key
openai.api_key = Secret

def instruction_based_prompting(question, inquiry_type):
    instructions = {
        "technical": "Provide a detailed step-by-step solution to the technical issue described.",
        "billing": "Explain the billing process clearly and address any concerns related to charges or payments.",
        "general": "Offer a concise and helpful response to the general customer inquiry."
    }

    instruction = instructions.get(inquiry_type, "Provide a helpful response.")

    prompt = f"{instruction}\nQuestion: {question}"

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  
        messages=[
            {"role": "system", "content": "You are a helpful customer support assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=150,  
        temperature=0.7  
    )

    return response.choices[0].message['content'].strip()

customer_inquiries = [
    ("How can I reset my password?", "technical"),
    ("What is the refund policy?", "billing"),
    ("How do I contact customer support?", "general")
]

for question, inquiry_type in customer_inquiries:
    response = instruction_based_prompting(question, inquiry_type)
    print(f"Question: {question}\nResponse: {response}\n")

Question: How can I reset my password?
Response: Here is a step-by-step guide on how you can reset your password:

1. Go to the login page of the website or application where you need to reset your password.
2. Look for the "Forgot Password" or "Reset Password" link. This is usually located near the login fields.
3. Click on the "Forgot Password" or "Reset Password" link. You may be redirected to a new page or a pop-up window.
4. On the password reset page, you will typically be asked to enter the email address associated with your account. Make sure to enter the correct email address.
5. After entering your email address, click on the "Reset Password" or "Send Reset Link" button. This will trigger an email

Question: What is the refund policy?
Response: Thank you for your question about our refund policy. Our refund policy typically depends on the type of product or service you have purchased. Here are some general guidelines:

1. Physical Products: If you have purchased a physical pr

### Role-Playing Prompting

Ask the model to respond as a customer service representative or technical support expert.
Evaluate how well the model adopts the role and provides relevant information.

In [16]:
import openai

openai.api_key = Secret

def role_playing_prompting(question, role):
    role_messages = {
        "customer_service": "You are a friendly and helpful customer service representative.",
        "technical_support": "You are a knowledgeable technical support expert."
    }

    role_message = role_messages.get(role, "You are a helpful assistant.")

    prompt = f"Please respond to the following inquiry: {question}"

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  
        messages=[
            {"role": "system", "content": role_message},  
            {"role": "user", "content": prompt}  
        ],
        max_tokens=150,  
        temperature=0.7  
    )

    return response.choices[0].message['content'].strip()

inquiries_and_roles = [
    ("How can I reset my password?", "technical_support"),
    ("What is the refund policy?", "customer_service"),
    ("How do I contact customer support?", "customer_service")
]

for question, role in inquiries_and_roles:
    response = role_playing_prompting(question, role)
    print(f"Role: {role}\nQuestion: {question}\nResponse: {response}\n")

Role: technical_support
Question: How can I reset my password?
Response: To reset your password, you typically need to follow these steps:

1. Visit the login page of the website or service where you need to reset your password.
2. Look for an option that says "Forgot password" or "Need help signing in" and click on it.
3. You may be asked to enter your email address or username associated with your account.
4. Check your email for a password reset link or instructions on how to reset your password.
5. Follow the provided instructions to create a new password.
6. Make sure to choose a strong and secure password that you can remember easily.
7. Once you have successfully reset your password, try logging in with your new credentials.

If you encounter any issues during the password reset process

Role: customer_service
Question: What is the refund policy?
Response: Thank you for reaching out! Our refund policy allows customers to request a refund within 30 days of purchase for any unused

### Contextual Prompting

Provide relevant context from previous email threads or the knowledge base before posing the main question.
Test if providing context improves the accuracy and relevance of the responses.

In [17]:
import openai

openai.api_key = Secret

def contextual_prompting(context, question):
    prompt = f"Context: {context}\n\nPlease answer the following question: {question}"

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  
        messages=[
            {"role": "system", "content": "You are a helpful customer support assistant."},  
            {"role": "user", "content": prompt}  
        ],
        max_tokens=150,  
        temperature=0.7  
    )

    return response.choices[0].message['content'].strip()

contextual_inquiries = [
    ("The customer has been experiencing issues with their internet connection for the past week.", "How can I troubleshoot my internet connection?"),
    ("The customer recently purchased a subscription and is unsure about the billing cycle.", "What is the refund policy?")
]

for context, question in contextual_inquiries:
    response = contextual_prompting(context, question)
    print(f"Context: {context}\nQuestion: {question}\nResponse: {response}\n")

Context: The customer has been experiencing issues with their internet connection for the past week.
Question: How can I troubleshoot my internet connection?
Response: I'm sorry to hear that you've been having trouble with your internet connection. Here are some steps you can take to troubleshoot the issue:

1. Check all the physical connections - make sure all cables are securely plugged in.
2. Restart your modem and router by unplugging them for about 30 seconds and then plugging them back in.
3. Check if other devices are also experiencing internet issues to determine if the problem is with a specific device or the network.
4. Run a speed test to check your internet connection speed.
5. Contact your internet service provider to see if there are any known outages in your area or if they can assist you in diagnosing the issue further.

If you've tried these steps and are still experiencing

Context: The customer recently purchased a subscription and is unsure about the billing cycle.


### Conversational Prompting

Create a dialogue-style prompt where the model continues an ongoing conversation with the customer.
Observe how well the model maintains context and coherence in multi-turn conversations.

In [18]:
import openai

openai.api_key = Secret

def conversational_prompting(conversation_history, new_question):
    conversation_history.append({"role": "user", "content": new_question})

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  
        messages=conversation_history,  
        max_tokens=150,  
        temperature=0.7  
    )

    return response.choices[0].message['content'].strip()

conversation_history = [
    {"role": "system", "content": "You are a helpful customer support assistant."},
    {"role": "user", "content": "Hi, I'm having trouble with my internet connection."},
    {"role": "assistant", "content": "I'm sorry to hear that. Have you tried restarting your router?"}
]

new_question = "Yes, I have. What should I do next?"

response = conversational_prompting(conversation_history, new_question)
print(f"New Question: {new_question}\nResponse: {response}\n")

New Question: Yes, I have. What should I do next?
Response: If restarting the router didn't help, you can try checking the physical connections to make sure everything is properly plugged in. If that doesn't work, you may want to contact your internet service provider to see if there are any known outages or issues in your area. They may also be able to troubleshoot the problem with you over the phone.



### Contrastive Prompting

Show the model examples of both good and bad responses.
Use these contrasting examples to guide the model towards generating better responses.
Compare the results with other techniques.

In [19]:
import openai

openai.api_key = Secret

def contrastive_prompting(question):
    examples = [
        {"role": "user", "content": "How can I reset my password?"},
        {"role": "assistant", "content": "Good: To reset your password, go to the login page and click on 'Forgot Password'. Follow the instructions to reset your password."},
        {"role": "assistant", "content": "Bad: Just try to remember it."},
        {"role": "user", "content": "What is the refund policy?"},
        {"role": "assistant", "content": "Good: Our refund policy allows you to return products within 30 days of purchase for a full refund. Please ensure the product is in its original condition."},
        {"role": "assistant", "content": "Bad: No refunds."}
    ]

    examples.append({"role": "user", "content": question})

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  
        messages=[
            {"role": "system", "content": "You are a helpful customer support assistant. Learn from the examples of good and bad responses."}
        ] + examples,
        max_tokens=150,  
        temperature=0.7  
    )

    return response.choices[0].message['content'].strip()

new_questions = [
    "How do I contact customer support?",
    "Can I change my subscription plan?"
]

for question in new_questions:
    response = contrastive_prompting(question)
    print(f"Question: {question}\nResponse: {response}\n")

Question: How do I contact customer support?
Response: Good: You can reach our customer support team by phone at [phone number], by email at [email address], or through the contact form on our website. We are available Monday to Friday from 9am to 5pm.

Question: Can I change my subscription plan?
Response: Good: Yes, you can change your subscription plan at any time. Please log in to your account and navigate to the subscription section to make changes.



### Specificity Prompting

Ask the model to respond with a specific style, tone, or level of detail, such as formal, friendly, or concise.
Assess how well the model adapts its responses to the specified requirements.

In [20]:
import openai

openai.api_key = Secret

def specificity_prompting(question, style):
    style_instructions = {
        "formal": "Respond in a formal and professional manner.",
        "friendly": "Respond in a warm and friendly tone.",
        "concise": "Provide a brief and to-the-point response."
    }

    instruction = style_instructions.get(style, "Provide a helpful response.")

    prompt = f"{instruction}\nQuestion: {question}"

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  
        messages=[
            {"role": "system", "content": "You are a helpful customer support assistant."},  
            {"role": "user", "content": prompt}  
        ],
        max_tokens=150,  
        temperature=0.7  
    )

    return response.choices[0].message['content'].strip()

questions_and_styles = [
    ("How can I reset my password?", "formal"),
    ("What is the refund policy?", "friendly"),
    ("How do I contact customer support?", "concise")
]

for question, style in questions_and_styles:
    response = specificity_prompting(question, style)
    print(f"Style: {style}\nQuestion: {question}\nResponse: {response}\n")

Style: formal
Question: How can I reset my password?
Response: To reset your password, please visit the login page and click on the "Forgot Password" link. You will then be prompted to enter your email address associated with your account, and further instructions will be sent to you via email on how to reset your password. If you encounter any difficulties during the process, please do not hesitate to contact us for further assistance. Thank you.

Style: friendly
Question: What is the refund policy?
Response: Hello! Thank you for reaching out. Our refund policy allows for a full refund within 30 days of purchase if you are not satisfied with your product. If you have any questions or need further assistance, feel free to let us know. We're here to help!

Style: concise
Question: How do I contact customer support?
Response: You can contact customer support by phone, email, or live chat. You can find the contact information on the company's website.



### Iterative Refinement Prompting

Ask the model to refine or improve upon its previous response.
Experiment with multiple iterations to see if responses improve over time.